In [24]:
import pandas as pd
import json
import reportlab

In [42]:
path = 'noTimeCriticality/routing_output.json'
path2 = 'noTimeCriticality/sim_preprocessed_config.json'

routing_data_json = open(path)
routing_data_json = json.load(routing_data_json)

input_data_json = open(path2)
input_data_json = json.load(input_data_json)

In [62]:
# Extracting data from the JSON structure
routing_data = []
for day in routing_data_json["days"]:
    day_routes = []
    for vehicle in day["vehicles"]:
        day_routes.append(vehicle["route"])
    routing_data.append(day_routes)

# Creating a pandas DataFrame
df = pd.DataFrame(routing_data, columns=[f"Vehicle_{i+1}" for i in range(len(routing_data[0]))])
df = df.applymap(lambda x: [394 if elem == 395 else elem for elem in x])


In [63]:
df

,Vehicle_1,Vehicle_2,Vehicle_3,Vehicle_4,Vehicle_5,Vehicle_6,Vehicle_7,Vehicle_8,Vehicle_9
0,[],[],[],[],"[394, 129, 394]",[],[],[],"[394, 122, 394]"
1,[],[],[],[],[],[],[],[],[]
2,[],[],[],[],[],[],[],[],"[394, 103, 394]"
3,[],[],[],[],[],[],[],[],[]
4,[],[],[],[],[],[],[],[],[]
...,...,...,...,...,...,...,...,...,...
246,[],[],[],[],"[394, 182, 394]",[],[],"[394, 45, 394]",[]
247,[],[],[],[],"[394, 182, 394]",[],[],[],"[394, 311, 394]"
248,[],[],[],[],[],[],"[394, 349, 394]",[],"[394, 178, 394]"
249,[],[],[],[],[],[],[],[],[]


In [64]:
# Extract pickup sites data
pickup_sites_data = [
    {
        "location_index": site["location_index"],
        "latitude": site["lonlats"][1],
        "longitude": site["lonlats"][0]
    }
    for site in input_data_json["pickup_sites"]
]

# Extract depots data
depots_data = [
    {
        "location_index": 394,
        "latitude": depot["lonlats"][1],
        "longitude": depot["lonlats"][0]
    }
    for depot in input_data_json["depots"]
]

# Create DataFrames
pickup_sites_df = pd.DataFrame(pickup_sites_data)
depots_df = pd.DataFrame(depots_data)

coord_df = pd.concat([pickup_sites_df, depots_df], ignore_index = True)

In [65]:
coord_df

,location_index,latitude,longitude
0,0,60.945884,23.352659
1,1,60.905353,23.329982
2,2,60.913446,23.435890
3,3,60.987171,23.410918
4,4,60.956071,23.491162
...,...,...,...
390,390,60.595201,23.277416
391,391,61.266603,23.503156
392,392,60.887524,22.669414
393,393,60.588183,23.446406


In [98]:
merged_df = df.applymap(lambda route: [coord_df.loc[idx, ['latitude', 'longitude']].tolist() for idx in route])

In [99]:
merged_df

,Vehicle_1,Vehicle_2,Vehicle_3,Vehicle_4,Vehicle_5,Vehicle_6,Vehicle_7,Vehicle_8,Vehicle_9
0,[],[],[],[],"[[60.929833, 23.369694], [60.83979769075367, 2...",[],[],[],"[[60.929833, 23.369694], [61.036504853800366, ..."
1,[],[],[],[],[],[],[],[],[]
2,[],[],[],[],[],[],[],[],"[[60.929833, 23.369694], [60.99944150606829, 2..."
3,[],[],[],[],[],[],[],[],[]
4,[],[],[],[],[],[],[],[],[]
...,...,...,...,...,...,...,...,...,...
246,[],[],[],[],"[[60.929833, 23.369694], [60.71122680641582, 2...",[],[],"[[60.929833, 23.369694], [60.89594067989363, 2...",[]
247,[],[],[],[],"[[60.929833, 23.369694], [60.71122680641582, 2...",[],[],[],"[[60.929833, 23.369694], [61.02120331743758, 2..."
248,[],[],[],[],[],[],"[[60.929833, 23.369694], [60.779703334185186, ...",[],"[[60.929833, 23.369694], [60.6542093588023, 23..."
249,[],[],[],[],[],[],[],[],[]


In [108]:
# Step 1: Create a boolean mask to identify rows with empty lists in all columns
empty_rows_mask = df.apply(lambda row: all(isinstance(cell, list) and len(cell) == 0 for cell in row), axis=1)

# Step 2: Use the boolean mask to get the indexes of rows with empty lists
empty_row_indexes = (empty_rows_mask[empty_rows_mask].index+1).tolist()

# Step 3: Count the number of empty rows
num_empty_rows = len(empty_row_indexes)

# Step 4: Print the indexes and count
print("No routes for days:", empty_row_indexes)
print("Days of no driving required:", num_empty_rows)

No routes for days: [2, 4, 5, 6, 8, 13, 22, 32, 42, 74, 115, 120, 127, 132, 142, 147, 158, 176, 188, 202, 212, 220, 222, 228, 231, 235, 240, 244, 250]
Days of no driving required: 29


In [111]:
df_drive_days_only = merged_df[~empty_rows_mask]


In [110]:
df_drive_days_only

,Vehicle_1,Vehicle_2,Vehicle_3,Vehicle_4,Vehicle_5,Vehicle_6,Vehicle_7,Vehicle_8,Vehicle_9
0,[],[],[],[],"[[60.929833, 23.369694], [60.83979769075367, 2...",[],[],[],"[[60.929833, 23.369694], [61.036504853800366, ..."
2,[],[],[],[],[],[],[],[],"[[60.929833, 23.369694], [60.99944150606829, 2..."
6,[],[],[],"[[60.929833, 23.369694], [60.91501806870317, 2...","[[60.929833, 23.369694], [60.613732475651986, ...",[],[],[],[]
8,[],[],[],[],"[[60.929833, 23.369694], [60.86093280784229, 2...",[],[],[],[]
9,[],[],"[[60.929833, 23.369694], [61.02218435895387, 2...",[],[],[],[],[],[]
...,...,...,...,...,...,...,...,...,...
245,[],[],[],[],[],"[[60.929833, 23.369694], [60.6542093588023, 23...","[[60.929833, 23.369694], [61.020564434293306, ...",[],[]
246,[],[],[],[],"[[60.929833, 23.369694], [60.71122680641582, 2...",[],[],"[[60.929833, 23.369694], [60.89594067989363, 2...",[]
247,[],[],[],[],"[[60.929833, 23.369694], [60.71122680641582, 2...",[],[],[],"[[60.929833, 23.369694], [61.02120331743758, 2..."
248,[],[],[],[],[],[],"[[60.929833, 23.369694], [60.779703334185186, ...",[],"[[60.929833, 23.369694], [60.6542093588023, 23..."
